<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex14notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex14notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


---
## 準備
---


今回の notebook で行う実験の中には，それなりに実行時間が長くなるものもある．GPU を利用した方が短い時間で済ませられるので，次のようにランタイムのタイプを変更しよう．

1. Colab のメニューから「ランタイム」>「ランタイムのタイプを変更」>「CPU」に代えて「T4 GPU」を選択して「保存」
1. ランタイムが初期化されるので，必要なら全てのコードセルを最初から実行し直す


In [ ]:
# @title
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split

# NumPy の 疑似乱数生成器（rng = random number generator）
from numpy.random import default_rng
rng = default_rng() # 疑似乱数生成器を初期化

# PyTorch 関係のほげ
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchsummary

次のコードセルを実行して，`cuda` と表示されれば， PyTorch で GPU が使える状態になっている．`cpu` と表示される場合は，GPU が使えるようになっておらず，PyTorch のプログラムは CPU で実行される．


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

---
## 非線形オートエンコーダ
---

---
### 非線形オートエンコーダとは

非線形変換をはさむ

---
### 実験: 非線形AEによる手書き数字画像の次元圧縮，再構成,生成

#### データの準備

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw[:20000] / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw[:20000].astype(int) # クラスラベル．0 から 9 の整数値

# 学習データとテストデータの分割
XL, XT, yL, yT = train_test_split(Xall, yall, test_size=4000, random_state=4649, stratify=yall)
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)
NL, D = XL.shape
NT = len(XT)

#K = 10

# 平均を引いたデータを用意
Xm = np.mean(XL, axis=0)
XL2 = XL - Xm
XT2 = XT - Xm

In [ ]:
# 学習データの最初の50枚を可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        img = XL[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()

In [ ]:
# データを扱うためのクラス
#
class MMDataset(Dataset):

    def __init__(self, dataX):
        self.X = dataX

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        return X

#### 非線形AEの定義

In [ ]:
# 非線形オートエンコーダ
#
class NonlinearAE(nn.Module):

    def __init__(self, dimX, dimHidden, dimZ):
        super(NonlinearAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(dimX, dimHidden, bias=True), nn.ReLU(),
            nn.Linear(dimHidden, dimZ, bias=True),
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(dimZ, dimHidden, bias=True), nn.ReLU(),
            nn.Linear(dimHidden, dimX, bias=True),
        )

    def forward(self, X):
        Z = self.encoder(X)
        Xt = self.decoder(Z)
        return Xt

    def loss(self, Xt, X):
        return F.mse_loss(Xt, X, reduction='sum')

#### 学習等のための関数の定義

In [ ]:
# 学習の関数
#
def train(model, optimizer, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Xt = model(X)         # 一つのバッチ X を入力して出力 Xt を計算
        loss = model.loss(Xt, X) # 入力 X を正解として loss を計算
        optimizer.zero_grad()   # 勾配をリセット
        loss.backward()         # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()         # パラメータを更新
        n += len(X)
        loss_sum += loss.item()  # 損失関数の値

    return loss_sum/n

# 損失関数の値を求める関数
#
@torch.no_grad()
def evaluate(model, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Xt = model(X)           # 一つのバッチ X を入力して出力 Xt を計算
        loss = model.loss(Xt, X)  # 入力 X を正解として loss を計算
        n += len(X)
        loss_sum += loss.item() # 損失関数の値

    return loss_sum/n

#### 学習

In [ ]:
# データ読み込みの仕組みを作る
dsL = MMDataset(XL2)
dsT = MMDataset(XT2)
dlL = DataLoader(dsL, batch_size=100, shuffle=True)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

In [ ]:
# ネットワークモデルの定義
H = 10
net = NonlinearAE(D, 1000, H).to(device)

# パラメータ最適化器の設定
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# 学習の繰り返し回数
nepoch = 30

# ネットワークの構造を表示
torchsummary.summary(net, (1, D))

# 学習
L = []
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT')
for t in range(1, nepoch+1):
    lossL = train(net, optimizer, dlL) / D
    lossT = evaluate(net, dlT) / D
    L.append([t, lossL, lossT])
    if (t < 10) or (t % 10 == 0):
        print(f'{t}   {lossL:.5f}   {lossT:.5f}')

# 学習曲線の表示
data = np.array(L)
#fig, ax = plt.subplots(1, 1, facecolor='white', figsize=(, 4))
fig, ax = plt.subplots(1, 1)
ax.plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax.plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax.axhline(0.0, color='gray')
ax.legend()
ax.set_title(f'loss')
plt.show()

# 学習後の損失と識別率
loss2 = evaluate(net, dlL) / D
print(f'# lossL: {loss2:.5f}', end='   ')
loss2 = evaluate(net, dlT) / D
print(f'# lossT: {loss2:.5f}')

参考: PCA の再構成誤差（再構成次元数 vs 平均二乗誤差）
```
10 0.034308
20 0.023815
30 0.017970
40 0.014241
50 0.011672
```

#### 再構成

In [ ]:
# テストデータ1バッチ分の再構成
for i, X in enumerate(dlT):
    X = X.to(device)
    Xt = net(X)
    break
XX    = X.to('cpu').detach().numpy() + Xm
XXrec = Xt.to('cpu').detach().numpy() + Xm

# 再構成したテストデータの最初の10枚を可視化
ncol = 10
fig, ax = plt.subplots(2, ncol, figsize=(0.8*ncol, 0.8*2))

# 元画像
for j in range(ncol):
    img = XX[j, ::].reshape((28, 28))
    ax[0, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[0, j].axis('off')

# 再構成した画像
for j in range(ncol):
    img = XXrec[j, ::].reshape((28, 28))
    ax[1, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[1, j].axis('off')

fig.tight_layout()
plt.show()

# 再構成誤差
mse = np.mean((XX[:ncol] - XXrec[:ncol])**2)
print(f'MSE = {mse:.5f}')

#### 生成

In [ ]:
# 学習データの特徴量を求める
Z = np.empty((len(dsL), H))
n = 0
for i, X in enumerate(dlL):
    X = X.to(device)
    Z[n:n+len(X)] = net.encoder(X).cpu().detach().numpy()
    n += len(X)

# その平均と分散共分散行列
Zmu = np.mean(Z, axis=0)
Zcov = np.cov(Z.T)
print(Zmu.shape, Zcov.shape)

In [ ]:
# 生成
Zgen = rng.multivariate_normal(Zmu, Zcov, 50)
Zgen = torch.tensor(Zgen.astype(np.float32)).to(device)
XXrec = net.decoder(Zgen)
XXrec = XXrec.cpu().detach().numpy() + Xm
XXrec.shape

# 生成した画像50枚を可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        img = XXrec[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()

---
### 非線形オートエンコーダの拡張

- CNN化する
- デノイジングオートエンコーダ (Denoising AE)
- etc.

---
### 非線形オートエンコーダの応用

- 特徴抽出器として...
- 生成モデルとして...
- 異常検知...
- etc.

---
## 変分オートエンコーダ
---

**変分オートエンコーダ** (Variational AE, VAE)

この節の内容は，次の書籍に基づいています．より詳しく知りたいひとはそちらを参考にしてください：

「ゼロから作る Deep Learning (5) — 生成モデル編」 斎藤康毅，オライリージャパン，2024.

---
### VAE とは

ニューラルネットを用いる生成モデルの一種．連続な潜在変数 $\mathbf{z} \in R^{H}$ が非線形変換を受けて観測変数 $\mathbf{x} \in R^{D}$ となると考える．$\mathbf{z}$ から $\mathbf{x}$ への変換をニューラルネットでモデル化する．このニューラルネットは AE におけるデコーダに相当するので，VAE でもデコーダと呼ぶ．

VAE では，潜在変数 $\mathbf{z}$ は，ある決まった（パラメータ固定の）正規分布に従うと仮定する．
具体的には，平均 $\mathbf{0}$，分散共分散行列が単位行列 $I$ の正規分布に従うものとする．
つまり，

$$
p(\mathbf{z}) = N(\mathbf{z}; \mathbf{0}, I)
$$

この正規分布から得られたある値 $\mathbf{z}$ をデコーダに入力して得られる出力を

$$
\widehat{\mathbf{x}} = \textrm{Dec}(\mathbf{z}; \mathbf{\theta})
$$

と表すことにする．$\widehat{\mathbf{x}} \in R^{D}$ である．また，$\mathbf{\theta}$ は，デコーダのパラメータを表す．

このとき，観測変数 $\mathbf{x}$ は，平均が $\widehat{\mathbf{x}}$ で分散共分散行列が $I$ の正規分布に従うと仮定する．つまり，

$$
p(\mathbf{x}) = N(\mathbf{x}; \widehat{\mathbf{x}}, I)
$$

これが，VAEによるデータ生成のモデルである．VAE では，個々のデータは次の過程によって生成される．

1. 平均 $0$ 分散共分散行列 $I$ の正規分布からひとつの値 $\mathbf{z}$ を得る
1. それがデコーダによって変換されて $\widehat{\mathbf{x}}$ になる
1. 平均 $\widehat{\mathbf{x}}$ 分散共分散行列 $I$ の正規分布からひとつの値 $\mathbf{x}$ を得る



---
### VAE の学習

VAE の学習は，正規分布やGMMと同様に，対数尤度の最大化によって行う．

学習データを $\{ \mathbf{x}_n \}_{n = 1}^N$ とすると，VAE モデルの対数尤度
は次のような式となる．

$$
\sum_{n=1}^{N} \log p_{\mathbf{\theta}}(\mathbf{x}_n) = \sum_{n=1}^{N} \log \int p_{\mathbf{\theta}} (\mathbf{x}_n, \mathbf{z}) d\mathbf{z} = \sum_{n=1}^{N} \log \int p_{\mathbf{\theta}} (\mathbf{x}_n|\mathbf{z})p(\mathbf{z}) d\mathbf{z}
$$

第2項，第3項には連続な変数 $\mathbf{z}$ の積分が含まれているので，これらの式に基づいてパラメータの最適化を行うのは難しい．

そこで，EMアルゴリズムで考えたのと同様に，任意の確率分布 $q(\mathbf{z})$ を用いて式を変形する．以下，ひとつのデータに対する対数尤度 $\log p_{\mathbf{\theta}}(\mathbf{x})$ について考える（添え字 $n$ も省略）．

$$
\begin{aligned}
\log p_{\mathbf{\theta}}(\mathbf{x}) &=  \int q(\mathbf{z})\log \frac{p_{\mathbf{\theta}}(\mathbf{x}, \mathbf{z})}{q(\mathbf{z})} d\mathbf{z} + \int q(\mathbf{z}) \log \frac{q(\mathbf{z})}{p_{\mathbf{\theta}}(\mathbf{z}|\mathbf{x})} d\mathbf{z}\\
&= \textrm{ELBO} + D_{\textrm KL}(q(\mathbf{z})\Vert p_{\mathbf{\theta}}(\mathbf{z}|\mathbf{x}))
\end{aligned}
$$

式変形の過程は省略した（EMアルゴリズムの解説にある類似の式変形の $\sum$ を $\int$ に置き換えたものになっている）．

このように ELBO + KL-divergence の形になるので，EMアルゴリズムと同じ手続きで最適化を行うことも考えられるが，まだ困難がある（詳細は省略）．そこで，VAEでは，$q(\mathbf{z})$ を正規分布に限定し，その限定のもとで ELBO を最大化する，というアプローチをとる．
それでもまだ， $q(\mathbf{z})$ のパラメータ（平均と分散共分散行列）をデータごとに用意しないといけないという困難があるので，ニューラルネットを用いて，これらのパラメータをデータ $\mathbf{x}$ から求めることにする．

この役割を担うニューラルネットをエンコーダと呼ぶ．エンコーダは，$\mathbf{x}$ を入力すると，$\mathbf{x}$ に対応する $q(\mathbf{z})$ の平均と分散共分散行列を出力する．ただし，簡単のため， $q(\mathbf{z})$ の分散共分散行列は対角行列に限定する．このエンコーダは次式で表される．

$$
\mathbf{\mu}, \mathbf{\sigma} = \textrm{Enc}(\mathbf{x}; \mathbf{\phi})
$$

$\mathbf{\mu}$ は，$\mathbf{x}$ に対応する $q(\mathbf{z})$ の平均であり，$\mathbf{\sigma}$ は，$q(\mathbf{z})$ の分散共分散行列の対角要素の平方根（すなわち$\mathbf{z}$ の各要素の標準偏差）をならべたベクトルである（$\mathbf{\sigma} = (\sigma_1, \sigma_2, \ldots, \sigma_H)$）．また，$\mathbf{\phi}$ は，エンコーダのパラメータを表す．
$\mathbf{x}$ が与えられたときの $q(\mathbf{z})$ は，次式のように表される．

$$
q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x}) = N(\mathbf{z}; \mathbf{\mu}, \mathbf{\sigma}^2I)
$$

ここで，$\mathbf{\sigma}^2 I$ という式は，ベクトル $\sigma$ の要素ごとの2乗を対角要素にもつ対角行列を表している（が，数学的に正しい表現ではないので注意）．

VAEでは，以上のように定式化した上で，ELBO の最大化を行う．
$q$ はエンコーダでモデル化されたので，ELBO のパラメータは，デコーダのパラメータ $\mathbf{\theta}$ とエンコーダのパラメータ $\mathbf{\phi}$ である．あるデータ $\mathbf{x}$ のELBO の値を $\textrm{ELBO}(\mathbf{x}; \mathbf{\theta}, \mathbf{\phi})$ と表すと，これは次のように変形できる．


$$
\begin{aligned}
\textrm{ELBO}(\mathbf{x}; \mathbf{\theta}, \mathbf{\phi}) &=  \int q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})\log \frac{p_{\mathbf{\theta}}(\mathbf{x}, \mathbf{z})}{q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})} d\mathbf{z} =
\int q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})\log \frac{p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z})p(\mathbf{z})}{q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})} d\mathbf{z}\\
&= \int q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})\log p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z})d\mathbf{z} - \int q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})\log \frac{q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})}{p(\mathbf{z})} d\mathbf{z}\\
&= \underbrace{\textrm{E}_{q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})}[ \log p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z}) ]}_{J_1} - \underbrace{D_{\textrm KL}(q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x}) \Vert p(\mathbf{z}) )}_{J_2}\\
\end{aligned}
$$

上に示すように，最下行の2つの項を $J_1, J_2$ と表記する．



$J_1$ は，分布 $q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})$ のもとでの $\log p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z}) $ の期待値である．
コンピュータを用いてこのような期待値を求める場合，「モンテカルロ法」を利用して近似値を求めることがよく行われる．$J_1$ の近似値をモンテカロル法によって求める場合，分布 $q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})$ に従う乱数でいくつか $\mathbf{z}$ のサンプルを生成し，それらを用いて $\log p_{\mathbf{\theta}}(\mathbf{x}|\mathbf{z})$ の平均を求めることになる．
詳しく書くと次の手続きとなる：

1. $\mathbf{x}$ をエンコーダへ入力して， $\mathbf{\mu}$ と $\mathbf{\sigma}$ を得る．
1. $N(\mathbf{z}; \mathbf{\mu}, \mathbf{\sigma}^2I)$ に従う $S$ 個のサンプル $\mathbf{z}_1, \mathbf{z}_2, \ldots, \mathbf{z}_S$ を生成する．
1. それらをデコーダへ入力して，$\widehat{\mathbf{x}}_1, \widehat{\mathbf{x}}_2, \ldots, \widehat{\mathbf{x}}_S$ を得る．
1. $J_1 \approx \frac{1}{S} \sum_{s=1}^{S}\log N(\mathbf{x}|\widehat{\mathbf{x}}_s, I)$ とする．

ここでは $S$ 個のサンプルを用いるとしているが，VAEにおいては，実用上 $S=1$ で十分なことも多い．
その場合，

$$
\begin{aligned}
J_1 & \approx \log N(\mathbf{x}|\widehat{\mathbf{x}}_s, I) \\
&= \log \left( \frac{1}{\sqrt{(2\pi)^D |I|}} \exp \left(-\frac{1}{2}(\mathbf{x} - \widehat{\mathbf{x}})^{\top} I^{-1} (\mathbf{x} - \widehat{\mathbf{x}})\right) \right) \\
&= -\frac{1}{2}(\mathbf{x} - \widehat{\mathbf{x}})^{\top} (\mathbf{x} - \widehat{\mathbf{x}}) - \log \sqrt{(2\pi)^D} \\
&= -\frac{1}{2}\Vert \mathbf{x} - \widehat{\mathbf{x}} \Vert^2 - \frac{D}{2}\log{2\pi}
\end{aligned}
$$

となる．最下行の第2項は定数である．したがって，あるデータ $\mathbf{x}$ に対する $J_1$ の値は，そのデータをエンコーダ→デコーダに入力して得られる再構成 $\widehat{\mathbf{x}}$ との間の二乗誤差が小さくなればなるほど大きくなる．

一方，$J_2$ は，$q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})$ の $p(\mathbf{z})$ に対する KL-divergence である．
2つの分布は


$$
\begin{aligned}
q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x}) &= N(\mathbf{z}; \mathbf{\mu}, \mathbf{\sigma}^2I) \\
p(\mathbf{z}) &= N(\mathbf{z}; \mathbf{0}, I)
\end{aligned}
$$

であり，いずれも，分散共分散行列が対角行列の正規分布である．
ここで，
$p_A(\mathbf{z})$ が平均 $\mathbf{\mu}_A = (\mu_{A,1}, \mu_{A,2}, \ldots, \mu_{A,H})$ 分散共分散行列 $\textrm{diag}( \sigma_{A,1}^2, \sigma_{A,2}^2, \ldots, \sigma_{A,H}^2)$ の正規分布であり，
$p_B(\mathbf{z})$ が平均 $\mathbf{\mu}_B = (\mu_{B,1}, \mu_{B,2}, \ldots, \mu_{B,H})$ 分散共分散行列 $\textrm{diag}( \sigma_{B,1}^2, \sigma_{B,2}^2, \ldots, \sigma_{B,H}^2)$ の正規分布であるとき， $p_A(\mathbf{z})$ の $p_B(\mathbf{z})$ に対する KL-divergence は，次式のようになる（導出は省略）．

$$
D_{\textrm KL}(p_A(\mathbf{z}) \Vert p_B(\mathbf{z})) = -\frac{1}{2}\sum_{h=1}^{H} \left( 1 + \log\frac{\sigma_{A,h}^2}{\sigma_{B,h}^2} - \frac{(\mu_{A,h} - \mu_{B,h})^2}{\sigma_{B,h}^2} - \frac{\sigma_{A,h}^2}{\sigma_{B,h}^2} \right)
$$

したがって，

$$
J_2 = D_{\textrm KL}(q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x}) \Vert p(\mathbf{z}) ) = -\frac{1}{2}\sum_{h=1}^{H} (1 + \log{\sigma_h^2} - \mu_h^2 - \sigma_h^2)
$$

となる．$J_2$ の前に負号が付いているので，ELBO を最大化するためには，$J_2$ は小さければ小さいほどよい．つまり，$q_{\mathbf{\phi}}(\mathbf{z}|\mathbf{x})$ が $p(\mathbf{z})$ に近づけば近づくほど ELBO が大きくなる．


以上をまとめると，あるデータ $\mathbf{x}$ に対する ELBO の値は

$$
\textrm{ELBO}(\mathbf{x}; \mathbf{\theta}, \mathbf{\phi}) \approx -\frac{1}{2}\Vert \mathbf{x} - \widehat{\mathbf{x}} \Vert^2 + \frac{1}{2}\sum_{h=1}^{H} (1 + \log{\sigma_h^2} - \mu_h^2 - \sigma_h^2) + \mbox{const}
$$

となる．
したがって，学習データ $\{ \mathbf{x}_n \}_{n = 1}^N$ を用いて VAE を学習させるために用いる損失関数は，

$$
E(\mathbf{\theta}, \mathbf{\phi}) = \sum_{n=1}^N \Vert \mathbf{x}_n - \widehat{\mathbf{x}}_n \Vert^2 - \sum_{n=1}^N\sum_{h=1}^H \left( 1 + \log{\sigma_{n,h}^2} - \mu_{n,h}^2 - \sigma_{n,h}^2 \right)
$$

となる．ニューラルネットの学習は損失関数を最小化する形で定式化するのが一般的なので，ここでは $E = -2 \sum_n \textrm{ELBO}$ として損失関数を定義している（勾配法によるパラメータの最適化においては定数項は無関係なので無視している）．


### VAE の実装上の工夫

$D$ 次元のデータを $H$ 次元に変換してから再構成する非線形AEの場合，あるデータ $\mathbf{x}$ をネットワークに入力してその再構成 $\widehat{\mathbf{x}}$ を計算する過程は次の通りだった．

1. $\mathbf{x}$ をエンコーダに入力して $H$ 次元の出力 $\mathbf{z}$ を得る
1. $\mathbf{z}$ をデコーダに入力して $D$ 次元の出力（再構成） $\widehat{\mathbf{x}}$ を得る

VAE もエンコーダとデコーダから成るが，その計算過程は少し異なり，次のようになる．

1. $\mathbf{x}$ をエンコーダに入力して 2つの $H$ 次元ベクトル $\mathbf{\mu}$ と $\mathbf{\sigma}$ を得る
1. $N(\mathbf{z}; \mathbf{\mu}, \mathbf{\sigma}^2I)$ に従う乱数で $H$ 次元ベクトル $\mathbf{z}$ をひとつ生成する
1. $\mathbf{z}$ をデコーダに入力して $D$ 次元の出力（再構成） $\widehat{\mathbf{x}}$ を得る

上記の 1. の計算は，非線形AEのエンコーダの出力層に2倍の数の（$2H$個の）ニューロンを置いたもので実現できる．ただし，そのままでは $\sigma_1, \sigma_2, \ldots, \sigma_H$ は正でなければならないという制約条件を入れるのが難しい．
そこで，エンコーダには $\sigma_1, \sigma_2, \ldots, \sigma_H$ のかわりに $\log{\sigma_1^2}, \log{\sigma_2^2}, \ldots, \log{\sigma_H^2}$ を出力させるのが一般的である．これらの値は任意の実数値をとる．
$\log{\sigma_h^2}$ が得られたら， $\exp\left( \frac{1}{2}\log{\sigma_h^2} \right) = \sigma_h$ として $\sigma_h$ を得ることができる．

次に 2. の計算であるが，この計算の過程にはそのままでは勾配を計算できないところがあり，PyTorch などの深層学習フレームワークでうまく実装できない．そこで，次のように計算過程を変更する：

2-1. 標準正規分布に従う乱数を$H$個生成し，$\varepsilon_1, \varepsilon_2, \ldots, \varepsilon_H$ とする

2-2. $\mathbf{z}$ の要素 $z_h$ を $z_h = \mu_h + \sigma_h \varepsilon_h$ によって求める（$h = 1, 2, \ldots, H$）．

この計算によって得られる $z_h$ は平均 $\mu_h$ 分散 $\sigma_h$ の正規分布に従い，さらに勾配も計算可能となる．
この工夫を reparameterization trick という．

一方， 3. の計算は，非線形AEのデコーダと全く同じようにできる．

---
### 実験: VAEによる手書き数字画像の次元圧縮，再構成,生成


データの準備や一部の関数の定義は先の「非線形AEによる...」と共通．

#### VAE の定義

In [ ]:
### VAE Encoder
#
class VAEEncoder(nn.Module):

    def __init__(self, dimX, dimHidden, dimZ):
        super().__init__()
        self.layer1        = nn.Linear(dimX, dimHidden)
        self.layer2_mu    = nn.Linear(dimHidden, dimZ)
        self.layer2_logvar = nn.Linear(dimHidden, dimZ)

    def forward(self, X):
        Y = self.layer1(X)
        Y = F.relu(Y)
        mu    = self.layer2_mu(Y)
        logvar = self.layer2_logvar(Y)
        sigma = torch.exp(0.5*logvar)
        return mu, sigma


### VAE Decoder
#
class VAEDecoder(nn.Module):

    def __init__(self, dimZ, dimHidden, dimXt):
        super().__init__()
        self.layer1 = nn.Linear(dimZ, dimHidden)
        self.layer2 = nn.Linear(dimHidden, dimXt)

    def forward(self, Z):
        Y = self.layer1(Z)
        Y = F.relu(Y)
        Xt = self.layer2(Y)
        #Xt = F.sigmoid(Xt)
        return Xt

### reparameterization trick
#
def reparameterization(mu, sigma):
    eps = torch.randn_like(sigma)
    Z = mu + sigma * eps
    return Z


### VAE
#
class VariationalAE(nn.Module):

    def __init__(self, dimX, dimHidden, dimZ):
        super().__init__()
        self.encoder = VAEEncoder(dimX, dimHidden, dimZ)
        self.decoder = VAEDecoder(dimZ, dimHidden, dimX)

    def forward(self, X):
        mu, sigma = self.encoder(X)
        Z = reparameterization(mu, sigma)
        Xt = self.decoder(Z)
        return Xt, mu, sigma

    def reconstruct(self, X):
        mu, sigma = self.encoder(X)
        Xt = self.decoder(mu)
        return Xt

    def loss(self, Xt, X, mu, sigma):
        SQE = F.mse_loss(Xt, X, reduction='sum')
        sigma2 = sigma**2
        KLD = - torch.sum(1 + torch.log(sigma2) - mu**2 - sigma2)
        return SQE + KLD

#### 学習等のための関数の定義

In [ ]:
# 学習の関数
#
def trainVAE(model, optimizer, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Xt, mu, sigma = model(X) # 一つのバッチ X を入力して出力を計算
        loss = model.loss(Xt, X, mu, sigma) # 損失関数の値を計算
        optimizer.zero_grad()   # 勾配をリセット
        loss.backward()         # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()         # パラメータを更新
        n += len(X)
        loss_sum += loss.item()  # 損失関数の値

    return loss_sum/n


# 損失関数の値を求める関数
#
@torch.no_grad()
def evaluateVAE(model, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Xt, mu, sigma = model(X)  # 一つのバッチ X を入力して出力を計算
        loss = model.loss(Xt, X, mu, sigma) # 損失関数の値を計算
        n += len(X)
        loss_sum += loss.item() # 損失関数の値

    return loss_sum/n

#### 学習

In [ ]:
# データ読み込みの仕組みを作る
dsL = MMDataset(XL2)
dsT = MMDataset(XT2)
dlL = DataLoader(dsL, batch_size=100, shuffle=True)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

In [ ]:
# ネットワークモデルの定義
H = 10
vae = VariationalAE(D, 1000, H).to(device)

# パラメータ最適化器の設定
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

# 学習の繰り返し回数
nepoch = 30

# ネットワークの構造を表示
torchsummary.summary(vae, (1, D))

# 学習
L = []
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT')
for t in range(1, nepoch+1):
    lossL = trainVAE(vae, optimizer, dlL) / D
    lossT = evaluateVAE(vae, dlT) / D
    L.append([t, lossL, lossT])
    if (t < 10) or (t % 10 == 0):
        print(f'{t}   {lossL:.5f}   {lossT:.5f}')

# 学習曲線の表示
data = np.array(L)
fig, ax = plt.subplots(1, 1)
ax.plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax.plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax.axhline(0.0, color='gray')
ax.legend()
ax.set_title(f'loss')
plt.show()

# 学習後の損失と識別率
loss2 = evaluateVAE(vae, dlL) / D
print(f'# lossL: {loss2:.5f}', end='   ')
loss2 = evaluateVAE(vae, dlT) / D
print(f'# lossT: {loss2:.5f}')

#### 再構成

In [ ]:
# テストデータ1バッチ分の再構成
for i, X in enumerate(dlT):
    X = X.to(device)
    Xt1, mu, sigma = vae(X)
    Xt2, mu, sigma = vae(X)
    Xt3 = vae.reconstruct(X)
    break
XX     = X.to('cpu').detach().numpy() + Xm
XXrec1 = Xt1.to('cpu').detach().numpy() + Xm
XXrec2 = Xt2.to('cpu').detach().numpy() + Xm
XXrec3 = Xt3.to('cpu').detach().numpy() + Xm

# 再構成したテストデータの最初の10枚を可視化
ncol = 10
fig, ax = plt.subplots(4, ncol, figsize=(0.8*ncol, 0.8*4))

# 元画像
for j in range(ncol):
    img = XX[j, ::].reshape((28, 28))
    ax[0, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[0, j].axis('off')

# 再構成した画像（正規分布からサンプリング）
for j in range(ncol):
    img = XXrec1[j, ::].reshape((28, 28))
    ax[1, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[1, j].axis('off')

# 再構成した画像（正規分布からサンプリング）
for j in range(ncol):
    img = XXrec2[j, ::].reshape((28, 28))
    ax[2, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[2, j].axis('off')

# 再構成した画像（平均を使う）
for j in range(ncol):
    img = XXrec3[j, ::].reshape((28, 28))
    ax[3, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[3, j].axis('off')

fig.tight_layout()
plt.show()

# 再構成誤差
mse = np.mean((XX[:ncol] - XXrec3[:ncol])**2)
print(f'MSE = {mse:.5f}')

VAEの再構成は，データ $\mathbf{x}$ をエンコーダに入力して正規分布のパラメータを得る → その正規分布から $\mathbf{z}$ をランダムサンプリング → それをデコーダに入力して再構成 $\widehat{\mathbf{x}}$ を得る，という過程である．
そのため，同じ $\mathbf{x}$ に対しても $\widehat{\mathbf{x}}$ はランダムに揺らぐ．
図の上から1行目が $\mathbf{x}$ であり，2行目以降は3通りの再構成である．2行目と3行目はランダムサンプリングした2通りの $\mathbf{z}$ を用いた再構成，4行目は，エンコーダが出力した平均値をそのまま $\mathbf{z}$ として得られた再構成である．

#### 生成

In [ ]:
# 生成
Zgen = np.random.normal(size=50*H).reshape((50, H))
Zgen = torch.tensor(Zgen.astype(np.float32)).to(device)
XXrec = vae.decoder(Zgen)
XXrec = XXrec.cpu().detach().numpy() + Xm

# 生成した画像50枚を可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        img = XXrec[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()